<a href="https://colab.research.google.com/github/Amoghkori/steganography/blob/main/video_reveal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import keras
import cv2
import sys
import math
from keras.models import Model
from keras.models import load_model
from PIL import Image
import argparse
from keras.preprocessing.image import save_img
from skimage.util.shape import view_as_blocks


In [ ]:
def normalize_batch(imgs):
    return (imgs -  np.array([0.485, 0.456, 0.406])) /np.array([0.229, 0.224, 0.225])

# Denormalize output images                                                        
def denormalize_batch(imgs,should_clip=True):
    imgs= (imgs * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])
    
    if should_clip:
        imgs= np.clip(imgs,0,1)
    return imgs


In [ ]:
def shuffle(im, inverse = False):
  
  # Configure block size, rows and columns
  blk_size=56
  rows=np.uint8(img.shape[0]/blk_size)
  cols=np.uint8(img.shape[1]/blk_size)

  # Create a block view on image
  img_blks=view_as_blocks(im,block_shape=(blk_size,blk_size,3)).squeeze()
  img_shuff=np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)

  # Secret key maps
  map={0:2, 1:0, 2:3, 3:1}
  inv_map = {v: k for k, v in map.items()}

  # Perform block shuffling
  for i in range(0,rows):
    for j in range(0,cols):
     x,y = i*blk_size, j*blk_size
     if(inverse):
      img_shuff[x:x+blk_size, y:y+blk_size] = img_blks[inv_map[i],inv_map[j]]
     else:
      img_shuff[x:x+blk_size, y:y+blk_size] = img_blks[map[i],map[j]]
      
  return img_shuff

# Update progress bar
def update_progress(current_frame, total_frames):
    progress=math.ceil((current_frame/total_frames)*100)
    sys.stdout.write('\rProgress: [{0}] {1}%'.format('>'*math.ceil(progress/10), progress))
    

In [ ]:
model=load_model("/content/drive/MyDrive/models_mix/reveal2.h5",compile=False)

In [ ]:
import cv2
vidcap = cv2.VideoCapture("/content/container_video_out.mp4")
fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
out = cv2.VideoWriter('secret_video_out.mp4', fourcc, 20, (224,224))
while(vidcap.isOpened()):
  ret,img=vidcap.read()
  if ret==True:
    img1=cv2.resize(img,(224,224))
    cover = (cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)).reshape(1,224,224,3)/255.0
    secretout=model.predict([normalize_batch(cover)])
       
            # Postprocess secret image output
    secretout=denormalize_batch(secretout)
    secretout=np.squeeze(secretout)*255.0
    secretout=np.uint8(secretout)
    out.write(secretout)
    if cv2.waitKey(1) & 0xFF==ord("q"):
      break
  else:
    break
vidcap.release()
out.release()
cv2.destroyAllWindows()
